In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings, yaml, logging, pandas, time, numpy
sys.path.append("../src/") # go to parent dir
from main import prepare_data, evaluate_models
warnings.filterwarnings('ignore')
with open('../conf/log.yml', 'rt') as f:
    config = yaml.safe_load(f.read())
logging.config.dictConfig(config)

In [ ]:
%load_ext autoreload
%autoreload 2
x_columns = ['train', 'gare', 'date', 'arret', 'p2q0', 'p3q0', 'p4q0', 'p0q2', 'p0q3', 'p0q4']
x_columns = ['gare', 'arret', 'p2q0', 'p3q0', 'p4q0', 'p0q2', 'p0q3', 'p0q4']
x_train, x_test, y_train, y_test, x_to_predict = prepare_data(x_columns)

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from models.factory import ModelFactory
with open('../conf/models.yml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, x_train, y_train, x_test, y_test)
lines = []
for model in factory.get_models():
    configuration = configurations[model.name]
    if not configuration['class'] == 'models.keras_model.KerasModel':
        continue
    start = time.time()
    score = model.evaluate()
    end = time.time()
    duration = numpy.round((end-start), 2)
    logging.info(f'{model.name}={numpy.round(score, 4)} in {duration}s')
    lines.append({'name': model.name, 'score': score, 'duration': duration})
    
    history = model.history
    logging.info(history.history.keys())
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

df = pandas.DataFrame(lines).sort_values(by=['score'], ascending=True)

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas
x = pandas.concat([x_train, x_test])
y = pandas.concat([y_train, y_test])
evaluate_models(x, y, x, y, x_to_predict)